In [68]:
import pandas as pd
import requests
from arelle import Cntlr
import sqlalchemy as sa
from bs4 import BeautifulSoup
import re
import os
import numpy as np

In [2]:
con = sa.create_engine('sqlite:///edgar_htm_idx.db').connect()

In [9]:
# tickers = pd.read_excel('cik_ticker.xlsx')
# tickers.to_sql('tickercik',con,index=False)

In [3]:
con.execute('select * from sqlite_master').fetchall()

[('table', 'tickercik', 'tickercik', 601333, 'CREATE TABLE tickercik (\n\t"CIK" BIGINT, \n\t"Ticker" TEXT, \n\t"Name" TEXT, \n\t"Exchange" TEXT, \n\t"SIC" FLOAT, \n\t"Business" TEXT, \n\t"Incorporated" TEXT, \n\t"IRS" FLOAT\n)'),
 ('table', 'idx', 'idx', 2, 'CREATE TABLE idx (conm TEXT, type TEXT, cik TEXT, date TEXT, path TEXT, year INT, quarter INT)')]

In [297]:
F = pd.read_sql('select *, tc.* from idx inner join tickercik tc on tc.CIK=idx.cik where tc.Ticker="F" and type in ("10-K","10-Q")', con)

In [290]:
F['xbrl_path'] = F.path.apply(get_xbrl_path)

list index out of range
list index out of range


In [39]:
def get_xbrl_path(idx_path):
    try:
        resp = requests.get(idx_path)
        soup = BeautifulSoup(resp.content, 'lxml')
        href = soup('a',text=re.compile(r'.xml'))[0].get('href')
        return 'https://www.sec.gov'+href
    except Exception as e:
        print(e)
        return np.nan

In [292]:
def get_current_df(xbrl_path):
    # xbrl_path = get_xbrl_path(F.path.sample().iloc[0])
    arelle = Cntlr.Cntlr().modelManager.load(xbrl_path)

    arelle_df = pd.DataFrame(data=[(fact,
                        fact.value,
                        fact.concept.qname,
                        fact.context.hasSegment,
                        fact.context.period.viewText(),
                        fact.context.endDatetime)
                     for fact in arelle.facts],
                 columns=('Fact',       #Fact; We leave the fact in the table in case we want more out of it
                          'Value',      #Value, like 5,200,000 in "5.2m"
                          'Account',    #Account, like "Cash" in "Cash of 5.2m"
                          'Category',   #Category, like "for the parent company" in "Cash of 5.2m for the parent company"b
                          'Period',
                          'Time'))      #Time, like 2015/12/31 in "Cash of 5.2m for the parent company as of 2015"

    arelle_df['Start'] = pd.to_datetime(arelle_df.Period.str.slice(0,10),errors='coerce')
    arelle_df['End'] = pd.to_datetime(arelle_df.Period.str.slice(10,),errors='coerce')
    arelle_df['isPeriod'] = pd.notnull(arelle_df.End)
    arelle_df['Days'] = (arelle_df.End - arelle_df.Start).dt.days

    arelle_df = arelle_df[(arelle_df.Fact.map(lambda f: f.isNumeric and not f.isNil) & #Fact is Numeric (i.e. can be converted to a number)
    #                                                 arelle_df.Account.map(lambda a: a=='us-gaap:EarningsPerShareBasic') & #Just Cash; Account mentions "Cash", anywhere would give us too many values
                                                    arelle_df.Category.map(lambda c:not c))] # Could also use "~", which does boolean "not" on the entire column


    def get_contexts(arelle_df):
        top_days = arelle_df[arelle_df.isPeriod].Days.value_counts()
        top_periods = arelle_df[arelle_df.isPeriod].Period.value_counts()
        top_dates = arelle_df[~arelle_df.isPeriod].Period.value_counts()
        top_periods_df = top_periods.to_frame().reset_index()
        top_periods_df.columns = ['Period', 'count']
        top_periods_df['Start'] = pd.to_datetime(top_periods_df.Period.str.slice(0,10),errors='coerce')
        top_periods_df['End'] = pd.to_datetime(top_periods_df.Period.str.slice(10,),errors='coerce')
        top_periods_df['Days'] = (top_periods_df.End - top_periods_df.Start).dt.days
        current_days = top_days.index[0]
        current_period = top_periods_df[top_periods_df.Days==current_days].sort_values('End',ascending=False).iloc[0].Period
        current_date = top_dates.index[:2].sort_values(ascending=False)[0]
        current_date_compare = top_dates.index[:2].sort_values(ascending=False)[1]
        current_period_ly = current_period.replace(current_period[:4],str(int(current_period[:4])-1))
        current_date_ly = current_date.replace(current_date[:4],str(int(current_date[:4])-1))
        return current_period, current_date, current_date_compare, current_period_ly, current_date_ly

    period_categories_df = pd.DataFrame({'PeriodCategory':['current_period','current_date','current_date_compare','current_period_ly','current_date_ly'],'Period':get_contexts(arelle_df)})

    arelle_df = arelle_df.merge(period_categories_df, on='Period', how='outer')
    arelle_df.Account = arelle_df.Account.astype(str)

    current_period_df = arelle_df[(arelle_df.PeriodCategory=='current_period')&(arelle_df.Account.astype(str).str.contains('gaap'))&(~arelle_df.Category)]
    current_date_df = arelle_df[(arelle_df.PeriodCategory=='current_date')&(arelle_df.Account.astype(str).str.contains('gaap'))&(~arelle_df.Category)]
    current_df = pd.concat([current_period_df, current_date_df], ignore_index=True)
    
    return current_df

In [ ]:
%%time
df_currents = pd.DataFrame()
for xbrl_path in F.xbrl_path.iloc[2:]:
    print(xbrl_path)
    df_currents = df_currents.append(get_current_df(xbrl_path), ignore_index=True)

https://www.sec.gov/Archives/edgar/data/37996/000114036109017928/f-20090630.xml
https://www.sec.gov/Archives/edgar/data/37996/000114036109025042/f-20090930.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752310001218/f-20091231.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752310002965/f-20100331.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752310004856/f-20100630.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752310006707/f-20100930.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752311001210/f-20101231.xml
https://www.sec.gov/Archives/edgar/data/37996/000115752311002989/f-20110331.xml
https://www.sec.gov/Archives/edgar/data/37996/000003799611000004/f-20110630.xml
https://www.sec.gov/Archives/edgar/data/37996/000003799611000016/f-20110930.xml
https://www.sec.gov/Archives/edgar/data/37996/000003799612000007/f-20111231.xml
https://www.sec.gov/Archives/edgar/data/37996/000003799612000023/f-20120331.xml


In [295]:
df_currents

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
0,[],-2000000,us-gaap:OtherComprehensiveIncomeUnrealizedHold...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
1,[],0,us-gaap:IncomeLossFromDiscontinuedOperationsNe...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
2,[],28716000000,us-gaap:CostsAndExpenses,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
3,[],4073000000,us-gaap:ComprehensiveIncomeNetOfTaxIncludingPo...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
4,[],1722000000,us-gaap:OtherComprehensiveIncomeLossNetOfTaxPe...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
5,[],0.69,us-gaap:EarningsPerShareDiluted,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
6,[],23989000000,us-gaap:SalesRevenueNet,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
7,[],2346000000,us-gaap:IncomeLossFromContinuingOperationsIncl...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
8,[],3880000000,us-gaap:ComprehensiveIncomeNetOfTax,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period
9,[],193000000,us-gaap:ComprehensiveIncomeNetOfTaxAttributabl...,False,2009-04-012009-06-30,2009-07-01,2009-04-01,2009-06-30,True,90.0,current_period


In [284]:
current_period_df.Account.unique()

array(['us-gaap:AdvertisingExpense',
       'us-gaap:AssetRetirementObligationLiabilitiesSettled',
       'us-gaap:AssetRetirementObligationRevisionOfEstimate',
       'us-gaap:AvailableForSaleSecuritiesGrossRealizedGainsLossesSaleProceeds',
       'us-gaap:BusinessCombinationStepAcquisitionEquityInterestInAcquireeRemeasurementGainOrLoss',
       'us-gaap:CashFlowHedgeGainLossToBeReclassifiedWithinTwelveMonths',
       'us-gaap:CommonStockDividendsPerShareDeclared',
       'us-gaap:ComprehensiveIncomeNetOfTax',
       'us-gaap:ComprehensiveIncomeNetOfTaxAttributableToNoncontrollingInterest',
       'us-gaap:ComprehensiveIncomeNetOfTaxIncludingPortionAttributableToNoncontrollingInterest',
       'us-gaap:CostOfGoodsAndServicesSold', 'us-gaap:CostsAndExpenses',
       'us-gaap:CurrentFederalTaxExpenseBenefit',
       'us-gaap:CurrentForeignTaxExpenseBenefit',
       'us-gaap:CurrentIncomeTaxExpenseBenefit',
       'us-gaap:CurrentStateAndLocalTaxExpenseBenefit',
       'us-gaap:DeferredF

In [270]:
current_period_df[current_period_df.Account.str.contains('NetIncome')]

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
178,[],6598000000,us-gaap:NetIncomeLoss,False,2011-01-012011-09-30,2011-10-01,2011-01-01,2011-09-30,True,272.0,current_period
179,[],10000000,us-gaap:NetIncomeLossAttributableToNoncontroll...,False,2011-01-012011-09-30,2011-10-01,2011-01-01,2011-09-30,True,272.0,current_period
180,[],6682000000,us-gaap:NetIncomeLossAvailableToCommonStockhol...,False,2011-01-012011-09-30,2011-10-01,2011-01-01,2011-09-30,True,272.0,current_period


In [271]:
current_date_df.Account.unique()

array(['us-gaap:AccountsPayableCurrentAndNoncurrent',
       'us-gaap:AccumulatedOtherComprehensiveIncomeLossNetOfTax',
       'us-gaap:AdditionalPaidInCapitalCommonStock', 'us-gaap:Assets',
       'us-gaap:AssetsOfDisposalGroupIncludingDiscontinuedOperation',
       'us-gaap:CashAndCashEquivalentsAtCarryingValue',
       'us-gaap:CollateralAlreadyPostedAggregateFairValue',
       'us-gaap:DebtAndCapitalLeaseObligations',
       'us-gaap:DeferredIncomeTaxLiabilities',
       'us-gaap:DeferredTaxAssetsNet',
       'us-gaap:DerivativeFairValueOfDerivativeAsset',
       'us-gaap:DisposalGroupIncludingDiscontinuedOperationCashAndCashEquivalents',
       'us-gaap:EquityMethodInvestments', 'us-gaap:FIFOInventoryAmount',
       'us-gaap:FiniteLivedIntangibleAssetsNet',
       'us-gaap:InventoryFinishedGoods', 'us-gaap:InventoryLIFOReserve',
       'us-gaap:InventoryNet',
       'us-gaap:InventoryWorkInProcessAndRawMaterials',
       'us-gaap:Liabilities', 'us-gaap:LiabilitiesAndStockholdersEq

In [272]:
current_date_df[current_date_df.Account.str.contains('Cash')]

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
5,[],16460000000,us-gaap:CashAndCashEquivalentsAtCarryingValue,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
11,[],69000000,us-gaap:DisposalGroupIncludingDiscontinuedOper...,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
30,[],550000000,us-gaap:RestrictedCashAndCashEquivalents,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date


In [274]:
current_date_df[current_date_df.Account.str.contains('Assets')]

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
3,[],162740000000,us-gaap:Assets,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
4,[],502000000,us-gaap:AssetsOfDisposalGroupIncludingDisconti...,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
9,[],1694000000,us-gaap:DeferredTaxAssetsNet,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
14,[],100000000,us-gaap:FiniteLivedIntangibleAssetsNet,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
26,[],5977000000,us-gaap:OtherAssets,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date


In [273]:
current_date_df[current_date_df.Account.str.contains('Liabilities')]

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
8,[],1109000000,us-gaap:DeferredIncomeTaxLiabilities,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
19,[],156714000000,us-gaap:Liabilities,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
20,[],162740000000,us-gaap:LiabilitiesAndStockholdersEquity,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
21,[],109000000,us-gaap:LiabilitiesOfDisposalGroupIncludingDis...,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date


In [277]:
current_date_df[current_date_df.Account.str.contains('Stockholders')]

,Fact,Value,Account,Category,Period,Time,Start,End,isPeriod,Days,PeriodCategory
20,[],162740000000,us-gaap:LiabilitiesAndStockholdersEquity,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
32,[],5982000000,us-gaap:StockholdersEquity,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
33,[],6026000000,us-gaap:StockholdersEquityIncludingPortionAttr...,False,2011-09-30,2011-10-01,2011-09-30,NaT,False,NaN,current_date
